# Interaction with widgets and JS CallBacks

## Adding Widgets

Widgets are interactive controls that can be added to Bokeh applications to provide a front end user interface to a visualization. They can drive new computations, update plots, and connect to other programmatic functionality.
Widgets can be added directly to the document root or nested inside a layout. There are two ways to program a widget’s functionality:

- Use the `CustomJS` callback. This will work in standalone HTML documents.
- Use `bokeh serve` to start the Bokeh server and set up event handlers. *[DESCRIBED IN OTHER NOTEBOOKS]*

### Buttons

In [1]:
from bokeh.io import output_notebook, show
from bokeh.layouts import widgetbox
output_notebook()

Loading BokehJS ...

In [2]:
from bokeh.models.widgets import Button

button_warning = Button(label="Foo Warning", button_type="warning")
button_success = Button(label="Foo Success", button_type="success")

show(widgetbox(button_warning, button_success))

### Checkbox button group

In [3]:
from bokeh.models.widgets import CheckboxButtonGroup

checkbox_button_group = CheckboxButtonGroup(
        labels=["Option 1", "Option 2", "Option 3"], 
        active=[1, 2]) # active param show selected the slice you specify

show(widgetbox(checkbox_button_group))

### Checkbox group

In [4]:
from bokeh.models.widgets import CheckboxGroup

checkbox_group = CheckboxGroup(
        labels=["Option 1", "Option 2", "Option 3"], active=[0, 1])

show(widgetbox(checkbox_group))

## Data Table
Bokeh provides a sophisticated data table widget based on SlickGrid. Note that since the table is configured with a data source object, any plots that share this data source will automatically have selections linked between the plot and the table (even in static HTML documents). 

*The data table is sortable by clicking on the column name*

In [5]:
from datetime import date
from random import randint
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

data = dict(
        dates=[date(2017, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(widgetbox(data_table))

## Sliders

In [6]:
from bokeh.models.widgets import Slider, RangeSlider

slider = Slider(start=0, end=10, value=3, step=.1, title="Slider")
range_slider = RangeSlider(start=0, end=10, value=(5,9), step=.1, title="Range Slider")

show(widgetbox(slider, range_slider))

## Tab panes

In [7]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="red", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="blue", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

## JavaScript CallBacks
While the main goal of Bokeh is to provide a path to create rich interactive visualizations in the browser, purely from python, there will always be specialized use-cases that are outside the capabilities of the core library. For this reason, Bokeh provides different ways for users to supply custom JavaScript when necessary, so that users may add custom or specialized behaviours in response to property changes and other events.
To supply a snippet of JavaScript code that should be executed (in the browser) when some event occurs, use the `CustomJS model`:
```python
from bokeh.models.callbacks import CustomJS

callback = CustomJS(args=dict(xr=plot.x_range), code="""

// JavaScript code goes here

var a = 10;

// the model that triggered the callback is cb_obj:
var b = cb_obj.value;

// models passed as args are automagically available
xr.start = a;
xr.end = b;

""")
```

### CustomJS for Model Property Events¶
These CustomJS callbacks can be attached to property change events on any Bokeh model, using the `js_on_change` method of Bokeh models:

In [8]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

The above example shows how to attach a CustomJS callback to a Slider widget, so that whenever the slider value updates, the callback is executed to update some data.

### CustomJS with a Python function
A CustomJS callback can also be implemented as a Python function, which is then translated to JavaScript using PyScript. This makes it easier for users to define client-side interactions without having to learn JavaScript. 

To use this functionality you need the Flexx library (install with `conda install -c bokeh flexx` or `pip install flexx`).

In [9]:
## NOTE: This requires flexx or pscript installed: conda install -c conda-forge pscript

from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.05 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
        y[i] = window.Math.pow(x[i], f)
    source.change.emit()

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)

RuntimeError: To use Python functions for CustomJS, you need PScript ("conda install -c conda-forge pscript" or "pip install pscript")

### CustomJS for Selections
Bokeh also provides the means to specify the same kind of callback to be executed whenever a selection changes. As a simple demonstration, the example below simply copies selected points on the first plot to the second. However, more sophisticated actions and computations are easily constructed in a similar way.

In [ ]:
from random import random
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.callback = CustomJS(args=dict(s2=s2), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d1 = cb_obj.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)

layout = row(p1, p2)

show(layout)